#### NobelPrize

In [ ]:
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import time
import random
import re

# 设置头文件
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'Cookie':'_hjSessionUser_1004842=eyJpZCI6IjFlNGNjM2U0LWI5YzItNTQ4Ni1iMDU1LTIwYTQyY2E3ODFmMyIsImNyZWF0ZWQiOjE2OTI5NDQzMTk5NTQsImV4aXN0aW5nIjp0cnVlfQ==; _hjSession_1004842=eyJpZCI6IjU5ODQzYWMwLTM0NmYtNGViOC1hNDg4LWY4ODk2MWQ0OTkzMiIsImNyZWF0ZWQiOjE2OTMzNjQ1MzY3NTUsImluU2FtcGxlIjpmYWxzZX0=; _hjAbsoluteSessionInProgress=0; _gid=GA1.2.1201900062.1693364537; cookieNotice=1; _ga=GA1.2.1573572851.1692944320; _ga_MY66JBWQGX=GS1.1.1693364536.4.1.1693364835.24.0.0'
}

def get_people(url_list):
    
    data = pd.DataFrame(columns=['FIELD', 'TIME', 'NAME', 'LINK'])
    
    for url in url_list:
        # 获取获奖人数据
        requests.packages.urllib3.disable_warnings()
        html = requests.get(url, headers=headers, verify=False).text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 获取存储学校数据的tr标签
        a_list = soup.find_all('a')
        for a in a_list:
            if 'href' in a.attrs.keys() and re.match('https://www.nobelprize.org/prizes/(.*?)/(.*?)/(.*?)/facts/', a['href']):
                info = list(re.findall('https://www.nobelprize.org/prizes/(.*?)/(.*?)/(.*?)/facts/', a['href'], re.S)[0])
                FIELD = info[0]
                TIME = info[1]
                NAME = a.text.strip()
                LINK = a['href']
                value = [FIELD, TIME, NAME, LINK]
                data.loc[len(data.index)] = value
                print(value)    # 打印结果
                
    data.to_excel('NobelPrize(temp).xlsx', index=False)
    return data

def get_personal_info(data):
    # 建立数据表存储最终数据
    nobel = pd.DataFrame(columns=['FIELD', 'TIME', 'NAME', 'LINK', 'BORN_DATE', 'DIED_DATE','AFFILIATION', 'PRIZE_MOTIVATION', 'PRIZE_SHARE'])

    for i in range(len(data)):
        FIELD, TIME, NAME, LINK = data.iloc[i, 0], data.iloc[i, 1], data.iloc[i, 2], data.iloc[i, 3]
        requests.packages.urllib3.disable_warnings()
        # 访问页面并解析
        html = requests.get(LINK, headers=headers, verify=False).text
        soup = BeautifulSoup(html, 'html')

        try:
            content = soup.find('div', class_='content')
            ps = content.find_all('p')

            if len(ps)==5:
                BORN_DATE = ps[1].text.strip().replace('\n','')[6:]
                AFFILIATION = ps[2].text.strip().replace('\n','')[37:]
                PRIZE_MOTIVATION = ps[3].text[20:-2]
                PRIZE_SHARE = ps[4].text[13:]

                value = [FIELD, TIME, NAME, LINK, BORN_DATE, '', AFFILIATION, PRIZE_MOTIVATION, PRIZE_SHARE]

            elif len(ps)==6 or len(ps)==7:
                BORN_DATE = ps[1].text.strip().replace('\n','')[6:]
                DIED_DATE = ps[2].text.strip().replace('\n','')[6:]
                AFFILIATION = ps[3].text.strip().replace('\n','')[37:]
                PRIZE_MOTIVATION = ps[4].text[20:-2]
                PRIZE_SHARE = ps[5].text[13:]

                value = [FIELD, TIME, NAME, LINK, BORN_DATE, DIED_DATE, AFFILIATION, PRIZE_MOTIVATION, PRIZE_SHARE]

            nobel.loc[len(nobel.index)] = value

            print(value, 'success!')
            time.sleep(random.randint(1, 3))

        except:
            value = [FIELD, TIME, NAME, LINK]
            nobel.loc[len(nobel.index)] = value
            print(value, 'fail!')

url_list = ['https://www.nobelprize.org/prizes/lists/all-nobel-prizes-in-physics/',\
           'https://www.nobelprize.org/prizes/lists/all-nobel-prizes-in-chemistry/',\
           'https://www.nobelprize.org/prizes/lists/all-nobel-laureates-in-physiology-or-medicine/',\
           'https://www.nobelprize.org/prizes/lists/all-prizes-in-economic-sciences/',]
data = get_people(url_list)
nobel = get_personal_info(data)
nobel.to_excel('NobelPrize.xlsx', index=False)